# todo

1. download data and split the data based on the specs of the competition
2. import keras and create a basic model
3. create a better model
4. generate predictions
5. save predictions and submit to kaggle

# download statefarm data, split train,valid,sample

In [2]:
%cd /home/yns207/nbs/machine_learning/

/home/yns207/nbs/machine_learning


In [3]:
import os
CUR_DIR = os.getcwd()
DATA_DIR = os.path.join('/scratch', 'yns207', 'data_statefarm')
print(CUR_DIR, DATA_DIR)

/home/yns207/nbs/machine_learning /scratch/yns207/data_statefarm


In [27]:
%mkdir $DATA_DIR
%cd $DATA_DIR
!kg config -g -c 'state-farm-distracted-driver-detection'
!kg download

/scratch/yns207/data_statefarm
downloading https://www.kaggle.com/c/state-farm-distracted-driver-detection/download/sample_submission.csv.zip

sample_submission.csv.zip 100% |####################| Time: 0:00:00 465.9 KiB/s
downloading https://www.kaggle.com/c/state-farm-distracted-driver-detection/download/imgs.zip

imgs.zip 100% |#####################################| Time: 0:04:29  15.2 MiB/s
downloading https://www.kaggle.com/c/state-farm-distracted-driver-detection/download/driver_imgs_list.csv.zip

driver_imgs_list.csv.zip 100% |#####################| Time: 0:00:00 378.6 KiB/s


In [23]:
%cd $DATA_DIR
!unzip -q driver_imgs_list.csv.zip
!unzip -q imgs.zip
!unzip -q sample_submission.csv.zip

/scratch/yns207/data_statefarm


In [24]:
%ls -lah $DATA_DIR
%ls -lah $DATA_DIR/train | head -n 25
%ls -lah $DATA_DIR/test | head -n 25

total 4,1G
drwxr-x---  4 yns207 yns207 4,0K 16 mai   11:52 ./
drwx------  6 yns207 users  4,0K 13 mai   21:40 ../
-rw-r--r--  1 yns207 yns207 480K  7 avril  2016 driver_imgs_list.csv
-rw-r-----  1 yns207 yns207  93K 13 mai   21:44 driver_imgs_list.csv.zip
-rw-r-----  1 yns207 yns207 4,0G 13 mai   21:44 imgs.zip
-rw-r--r--  1 yns207 yns207 4,1M 29 mars   2016 sample_submission.csv
-rw-r-----  1 yns207 yns207 207K 13 mai   21:40 sample_submission.csv.zip
drwxr-xr-x  2 yns207 yns207 4,4M 31 déc.   2015 test/
drwxr-xr-x 12 yns207 yns207 4,0K 29 mars   2016 train/
total 1,4M
drwxr-xr-x 12 yns207 yns207 4,0K 29 mars   2016 ./
drwxr-x---  4 yns207 yns207 4,0K 16 mai   11:52 ../
drwxr-xr-x  2 yns207 yns207 152K 31 déc.   2015 c0/
drwxr-xr-x  2 yns207 yns207 124K 31 déc.   2015 c1/
drwxr-xr-x  2 yns207 yns207 132K 31 déc.   2015 c2/
drwxr-xr-x  2 yns207 yns207 132K 31 déc.   2015 c3/
drwxr-xr-x  2 yns207 yns207 128K 31 déc.   2015 c4/
drwxr-xr-x  2 yns207 yns207 144K 31 déc.   2015 c5/
drwxr-xr

In [25]:
%ls -lah $DATA_DIR/train/c0 | head -n 25

total 105M
drwxr-xr-x  2 yns207 yns207 152K 31 déc.   2015 ./
drwxr-xr-x 12 yns207 yns207 4,0K 29 mars   2016 ../
-rw-r--r--  1 yns207 yns207  49K 31 déc.   2015 img_100026.jpg
-rw-r--r--  1 yns207 yns207  37K 31 déc.   2015 img_10003.jpg
-rw-r--r--  1 yns207 yns207  40K 31 déc.   2015 img_100050.jpg
-rw-r--r--  1 yns207 yns207  40K 31 déc.   2015 img_100074.jpg
-rw-r--r--  1 yns207 yns207  40K 31 déc.   2015 img_10012.jpg
-rw-r--r--  1 yns207 yns207  45K 31 déc.   2015 img_100145.jpg
-rw-r--r--  1 yns207 yns207  40K 31 déc.   2015 img_100191.jpg
-rw-r--r--  1 yns207 yns207  43K 31 déc.   2015 img_100257.jpg
-rw-r--r--  1 yns207 yns207  28K 31 déc.   2015 img_100312.jpg
-rw-r--r--  1 yns207 yns207  38K 31 déc.   2015 img_100337.jpg
-rw-r--r--  1 yns207 yns207  42K 31 déc.   2015 img_100456.jpg
-rw-r--r--  1 yns207 yns207  46K 31 déc.   2015 img_10053.jpg
-rw-r--r--  1 yns207 yns207  40K 31 déc.   2015 img_100542.jpg
-rw-r--r--  1 yns207 yns207  38K 31 déc.   2015 img_100598.jpg
-rw-r--

In [26]:
%mkdir $DATA_DIR/test/uknown
%mv $DATA_DIR/test/*.jpg $DATA_DIR/test/uknown/

In [27]:
#load driver info when craeting validation set so that
#no one driver appears in both valid and train sets
#we want this to be the case becaus on kaggle they say that no
#one driver is both in the train and test sets and we want to mimic this

import pandas as pd
%cd $DATA_DIR
driver_imgs = pd.read_csv('driver_imgs_list.csv')

/scratch/yns207/data_statefarm


In [28]:
driver_imgs.head(10)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
5,p002,c0,img_3370.jpg
6,p002,c0,img_67639.jpg
7,p002,c0,img_58560.jpg
8,p002,c0,img_35779.jpg
9,p002,c0,img_10012.jpg


In [29]:
import random
num_users = len(driver_imgs['subject'].unique())
sample_users = random.sample(list(driver_imgs['subject'].unique()) ,5)
sample_users

['p047', 'p014', 'p035', 'p051', 'p075']

In [30]:
# make valid folder tree
%cd $DATA_DIR
!for i in {0..9}; do mkdir -p valid/c$i; done

/scratch/yns207/data_statefarm


In [31]:
# take the sample users and move all their images to valid
%cd $DATA_DIR
for index,row in driver_imgs[driver_imgs['subject'].isin(sample_users)].iterrows():
    old = os.path.join('train', row['classname'], row['img'])
    new = os.path.join('valid', row['classname'], row['img'])
    os.rename(old, new)

/scratch/yns207/data_statefarm


In [32]:
%cd $DATA_DIR
!for i in {0..9}; do echo c$i; ls -lah valid/c$i | wc -l; done

/scratch/yns207/data_statefarm
c0
540
c1
440
c2
438
c3
440
c4
447
c5
443
c6
456
c7
411
c8
320
c9
388


In [33]:
# now create a sample of the training and valid sets
%cd $DATA_DIR
!for i in {0..9}; do mkdir -p sample/train/c$i; mkdir -p sample/valid/c$i; done

/scratch/yns207/data_statefarm


In [34]:
import numpy as np
from shutil import copyfile
from glob import glob

%cd $DATA_DIR

g_train = np.random.permutation(glob(os.path.join('train', 'c?/*jpg')))
g_valid = np.random.permutation(glob(os.path.join('valid', 'c?/*jpg')))

for i in range(1500):
    copyfile(g_train[i], os.path.join(DATA_DIR, 'sample', g_train[i]))
for i in range(1000):
    copyfile(g_valid[i], os.path.join(DATA_DIR, 'sample', g_valid[i]))

/scratch/yns207/data_statefarm


In [35]:
%mkdir -p $DATA_DIR/results
%mkdir -p $DATA_DIR/sample/results $DATA_DIR/sample/test

# import keras and make a basic model

In [127]:
from theano.sandbox import cuda

In [162]:
%matplotlib inline
from IPython.display import FileLink

import h5py
import numpy as np
import PIL

from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras.regularizers import l2

In [144]:
import os
path = os.path.join(DATA_DIR, 'sample')
test_path = os.path.join(path, 'test')
models_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')
batch_size = 64

/scratch/yns207/data_statefarm/sample/train


In [146]:
gen = image.ImageDataGenerator()
tr_batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)
va_batches = gen.flow_from_directory(valid_path, target_size=(224,224), class_mode='categorical', shuffle=False, batch_size=batch_size*2)
# te_batches = gen.flow_from_directory(test_path, target_size=(224,224), class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [147]:
va_classes = va_batches.classes
tr_classes = tr_batches.classes
va_labels = to_categorical(va_classes)
tr_labels = to_categorical(tr_classes)
va_filenames = va_batches.filenames
tr_filenames = tr_batches.filenames
# te_filenames = te_batches.filenames

In [148]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,224,224)),
    Flatten(),
    Dense(10, activation='softmax')
])
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [149]:
model.fit_generator(tr_batches, 
                    steps_per_epoch=tr_batches.n//batch_size, 
                    validation_data=va_batches, 
                    validation_steps=va_batches.n//batch_size,
                    epochs=2)

Epoch 1/2
23/23 [==============================] - 119s - loss: 12.3475 - acc: 0.1760 - val_loss: 13.8419 - val_acc: 0.1366
Epoch 2/2
23/23 [==============================] - 70s - loss: 12.8302 - acc: 0.1975 - val_loss: 14.1261 - val_acc: 0.1234


In [150]:
'''
the model is stuck, its likely predicting 1-2 classes with high accuracy 
and giving all the other classes 0 to try to minimize its loss,
default learning rate on Adam optimizer is too large
'''
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 3, 224, 224)       12        
_________________________________________________________________
flatten_4 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1505290   
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
_________________________________________________________________


In [215]:
#try a lower learning rate at the start
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,224,224)),
    Flatten(),
    Dense(10, activation='softmax')
])
model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=2)

Epoch 1/2
24/24 [==============================] - 36s - loss: 5.0429 - acc: 0.2699 - val_loss: 9.5460 - val_acc: 0.2090
Epoch 2/2
24/24 [==============================] - 26s - loss: 1.5623 - acc: 0.6130 - val_loss: 5.6561 - val_acc: 0.2780


In [216]:
# ok now train with a slightly higher learning rate
model.optimizer.lr=0.001
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=4)

Epoch 1/4
24/24 [==============================] - 38s - loss: 0.5130 - acc: 0.8511 - val_loss: 4.0271 - val_acc: 0.3250
Epoch 2/4
24/24 [==============================] - 26s - loss: 0.2268 - acc: 0.9365 - val_loss: 3.4260 - val_acc: 0.3930
Epoch 3/4
24/24 [==============================] - 29s - loss: 0.1249 - acc: 0.9808 - val_loss: 2.9233 - val_acc: 0.3740
Epoch 4/4
24/24 [==============================] - 27s - loss: 0.0723 - acc: 0.9961 - val_loss: 2.7301 - val_acc: 0.3700


In [217]:
rand_batches = gen.flow_from_directory(valid_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size*2)

Found 1000 images belonging to 10 classes.


In [218]:
# use our random batches to show that were 
# hovering around .38 validation accuracy
# apparently an improvement of more tahn 3% here
# is 'non random'
val_res = [model.evaluate_generator(rand_batches, rand_batches.n // (batch_size*2)) for i in range(10)]
np.round(val_res, 2)

array([[ 2.7 ,  0.37],
       [ 2.7 ,  0.37],
       [ 2.72,  0.37],
       [ 2.67,  0.37],
       [ 2.79,  0.37],
       [ 2.71,  0.38],
       [ 2.76,  0.37],
       [ 2.72,  0.37],
       [ 2.74,  0.37],
       [ 2.75,  0.37]])

# make better model

In [231]:
# add l2 regularization to the model
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', kernel_regularizer=l2(0.01))
])

model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=2)

Epoch 1/2
24/24 [==============================] - 25s - loss: 4.8744 - acc: 0.2856 - val_loss: 9.5252 - val_acc: 0.2400
Epoch 2/2
24/24 [==============================] - 25s - loss: 2.9543 - acc: 0.6059 - val_loss: 4.5971 - val_acc: 0.3310


In [232]:
model.optimizer.lr=0.001
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=4)

Epoch 1/4
24/24 [==============================] - 32s - loss: 2.4035 - acc: 0.7617 - val_loss: 5.7990 - val_acc: 0.2490
Epoch 2/4
24/24 [==============================] - 28s - loss: 2.0709 - acc: 0.8482 - val_loss: 4.9713 - val_acc: 0.2910
Epoch 3/4
24/24 [==============================] - 31s - loss: 1.9489 - acc: 0.8860 - val_loss: 4.3251 - val_acc: 0.3500
Epoch 4/4
24/24 [==============================] - 30s - loss: 1.9589 - acc: 0.8888 - val_loss: 4.2250 - val_acc: 0.3550


In [184]:
# uh its worse meh.

In [240]:
# add a single hidden layer
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,224,224)),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=2)
model.optimizer.lr = 0.01
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=5)

Epoch 1/2
24/24 [==============================] - 89s - loss: 2.0835 - acc: 0.3756 - val_loss: 10.7966 - val_acc: 0.1530
Epoch 2/2
24/24 [==============================] - 82s - loss: 1.1836 - acc: 0.6766 - val_loss: 5.9594 - val_acc: 0.1850
Epoch 1/5
24/24 [==============================] - 85s - loss: 0.6481 - acc: 0.8827 - val_loss: 3.1254 - val_acc: 0.2730
Epoch 2/5
24/24 [==============================] - 59s - loss: 0.3647 - acc: 0.9573 - val_loss: 2.3699 - val_acc: 0.3190
Epoch 3/5
24/24 [==============================] - 78s - loss: 0.2061 - acc: 0.9894 - val_loss: 1.8070 - val_acc: 0.4280
Epoch 4/5
24/24 [==============================] - 78s - loss: 0.1329 - acc: 0.9920 - val_loss: 1.7206 - val_acc: 0.4520
Epoch 5/5
24/24 [==============================] - 58s - loss: 0.0835 - acc: 1.0000 - val_loss: 1.7591 - val_acc: 0.4670


In [196]:
# well that turned out ok, but were stuck at 100% training accuracy

In [241]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,(3,3), activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=2)
    model.optimizer.lr = 0.01
    model.fit_generator(tr_batches, 
                        steps_per_epoch=(tr_batches.n//batch_size)+1, 
                        validation_data=va_batches, 
                        validation_steps=(va_batches.n//batch_size)+1,
                        epochs=5)
    return model

In [242]:
conv1(tr_batches)

/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys


Epoch 1/2
24/24 [==============================] - 57s - loss: 2.2827 - acc: 0.2601 - val_loss: 2.5020 - val_acc: 0.1180
Epoch 2/2
24/24 [==============================] - 85s - loss: 1.2738 - acc: 0.6276 - val_loss: 2.3498 - val_acc: 0.1460
Epoch 1/5
24/24 [==============================] - 61s - loss: 0.7765 - acc: 0.8305 - val_loss: 2.3097 - val_acc: 0.1970
Epoch 2/5
24/24 [==============================] - 68s - loss: 0.4928 - acc: 0.9321 - val_loss: 2.3182 - val_acc: 0.1920
Epoch 3/5
24/24 [==============================] - 98s - loss: 0.3318 - acc: 0.9661 - val_loss: 2.3228 - val_acc: 0.1840
Epoch 4/5
24/24 [==============================] - 76s - loss: 0.2358 - acc: 0.9868 - val_loss: 2.3251 - val_acc: 0.1830
Epoch 5/5
24/24 [==============================] - 56s - loss: 0.1726 - acc: 0.9927 - val_loss: 2.2913 - val_acc: 0.1880


In [245]:
# add data augmentation image shimage.ImageDataGeneratorGenerator
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
tr_batches = gen_t.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [246]:
model = conv1(tr_batches)

/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys


Epoch 1/2
24/24 [==============================] - 64s - loss: 2.9113 - acc: 0.1138 - val_loss: 2.3154 - val_acc: 0.0900
Epoch 2/2
24/24 [==============================] - 61s - loss: 2.6070 - acc: 0.1599 - val_loss: 2.3083 - val_acc: 0.1220
Epoch 1/5
24/24 [==============================] - 133s - loss: 2.4072 - acc: 0.2010 - val_loss: 2.3247 - val_acc: 0.1310
Epoch 2/5
24/24 [==============================] - 65s - loss: 2.2894 - acc: 0.2406 - val_loss: 2.3689 - val_acc: 0.1080
Epoch 3/5
24/24 [==============================] - 67s - loss: 2.1034 - acc: 0.2990 - val_loss: 2.4052 - val_acc: 0.1020
Epoch 4/5
24/24 [==============================] - 55s - loss: 2.0475 - acc: 0.3269 - val_loss: 2.4341 - val_acc: 0.1030
Epoch 5/5
24/24 [==============================] - 67s - loss: 1.8624 - acc: 0.3710 - val_loss: 2.4401 - val_acc: 0.1160


In [247]:
model.optimizer.lr = 0.0001
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=5)

Epoch 1/5
24/24 [==============================] - 123s - loss: 1.9066 - acc: 0.3652 - val_loss: 2.4032 - val_acc: 0.1330
Epoch 2/5
24/24 [==============================] - 93s - loss: 1.7616 - acc: 0.4119 - val_loss: 2.3677 - val_acc: 0.1510
Epoch 3/5
24/24 [==============================] - 91s - loss: 1.6931 - acc: 0.4350 - val_loss: 2.2999 - val_acc: 0.1680
Epoch 4/5
24/24 [==============================] - 94s - loss: 1.6599 - acc: 0.4556 - val_loss: 2.2007 - val_acc: 0.2020
Epoch 5/5
24/24 [==============================] - 92s - loss: 1.5929 - acc: 0.4770 - val_loss: 2.1280 - val_acc: 0.2470


In [248]:
model.fit_generator(tr_batches, 
                    steps_per_epoch=(tr_batches.n//batch_size)+1, 
                    validation_data=va_batches, 
                    validation_steps=(va_batches.n//batch_size)+1,
                    epochs=25)

Epoch 1/25
24/24 [==============================] - 118s - loss: 1.4795 - acc: 0.5149 - val_loss: 2.0564 - val_acc: 0.3020
Epoch 2/25
24/24 [==============================] - 97s - loss: 1.4884 - acc: 0.5158 - val_loss: 1.9588 - val_acc: 0.3410
Epoch 3/25
24/24 [==============================] - 105s - loss: 1.4316 - acc: 0.5251 - val_loss: 1.8608 - val_acc: 0.3720
Epoch 4/25
24/24 [==============================] - 88s - loss: 1.4046 - acc: 0.5383 - val_loss: 1.8038 - val_acc: 0.3750
Epoch 5/25
24/24 [==============================] - 92s - loss: 1.3783 - acc: 0.5594 - val_loss: 1.7448 - val_acc: 0.4170
Epoch 6/25
24/24 [==============================] - 94s - loss: 1.3235 - acc: 0.5855 - val_loss: 1.6832 - val_acc: 0.4540
Epoch 7/25
24/24 [==============================] - 97s - loss: 1.3069 - acc: 0.5782 - val_loss: 1.6179 - val_acc: 0.4820
Epoch 8/25
24/24 [==============================] - 85s - loss: 1.2693 - acc: 0.5873 - val_loss: 1.5992 - val_acc: 0.4780
Epoch 9/25
24/24 [====